# Task 2: Recommendation Engine

### Setting up the Notebook

In [1]:
import numpy as np
import pandas as pd

from src.utils import read_csv
from src.data_preprocessor import DataPreprocessor
from src.preprocessor_utils import remove_columns, convert_to_lowercase

from src.recommendation_utils import get_recommendation_weights

In [2]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### Load the Data

In [3]:
trainX, trainY = read_csv('data/train.csv', ylabel='price')
# data_preprocessor = DataPreprocessor()
# trainX, trainY = data_preprocessor.fit_transform(trainX, trainY)

df = pd.concat([trainX, trainY], axis=1)

## Computing the Top Recommendations

In [10]:
def get_top_recommendations(row, df, k=3):

    ######## Get Recommendation Weights For Each Property
    df_weights = get_recommendation_weights(row, df)
    
    ######## Filter Out Top Properties (More Than Requested)
    top_property_indices = df_weights.argsort()[-10*k:]
    
    ######## Introduce Randomness in Choice From The Top Recommendations
    prob = df_weights[top_property_indices]
    prob = prob/prob.sum()
    rec_index = np.random.choice(df.index[top_property_indices], size=k, replace=False, p=prob)
    
    return rec_index

## Testing the Recommendation Engine

### Pick a Sample Listing as Input

In [11]:
# Pick a row id of choice
row_id = 10
#row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row = df.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
# data_preprocessor.inverse_transform(pd.DataFrame([row]))
pd.DataFrame([row])

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
10,397777,2 bed condo for sale in bedok court,299 bedok south avenue 3,bedok court,condo,99-year leasehold,1985.0,2.0,NaN,1733,...,unspecified,"studio, 1, 2, 3, 4 br",280.0,https://www.99.co/singapore/condos-apartments/...,1.322153,103.945223,0,bedok south,bedok,2205000.0


### Compute and Display the recommendations

In [12]:
k = 3

recommendation_list = get_top_recommendations(row, df, k=k)

# data_preprocessor.inverse_transform(df.iloc[recommendation_list])
df.iloc[recommendation_list]

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
5611,151758,1 bed condo for sale in eastwood regency,40 eastwood road,eastwood regency,Condo,freehold,2013.0,1.0,1.0,517,...,partial,"studio, 1 br",75.0,https://www.99.co/singapore/condos-apartments/...,1.321195,103.957529,0,bedok south,bedok,933300.0
12258,338342,4 bed condo for sale in bedok residences,22 bedok north drive,bedok residences,condo,99-year leasehold,2015.0,4.0,4.0,1970,...,unspecified,"studio, 1, 2, 3, 4 br",583.0,https://www.99.co/singapore/condos-apartments/...,1.321972,103.946825,0,bedok south,bedok,3024000.0
1721,645298,7 bed house for sale in english villas,bedok rise,english villas,Semi-Detached House,freehold,1999.0,7.0,5.0,7427,...,unspecified,"6, 7 br",11.0,https://www.99.co/singapore/houses/english-vil...,1.327341,103.949867,0,bedok south,bedok,4200000.0
